<a href="https://colab.research.google.com/github/Pennyvia/FutureIntern_AI_01/blob/main/FutureIntern_AI_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer

# Load dataset
dataset = load_dataset('wikitext', 'wikitext-2-raw-v1')

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the dataset
def tokenize_function(examples):
    inputs = tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128)
    inputs['labels'] = inputs['input_ids'].copy()
    return inputs

tokenized_datasets = dataset.map(tokenize_function, batched=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/4358 [00:00<?, ? examples/s]

Map:   0%|          | 0/36718 [00:00<?, ? examples/s]

Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [14]:

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorWithPadding,EarlyStoppingCallback
from datasets import load_dataset,load_metric
import numpy as np

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define a data collator
data_collator = DataCollatorWithPadding(tokenizer)

# Load dataset
dataset = load_dataset('wikitext', 'wikitext-2-raw-v1')

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('gpt2')
model = AutoModelForCausalLM.from_pretrained('gpt2').to(device)

# Set the EOS token as the padding token
tokenizer.pad_token = tokenizer.eos_token

#define compute metrics
def compute_metrics(eval_pred):
    metric = load_metric("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


# Tokenize the dataset
def tokenize_function(examples):
    inputs =  tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128)
    inputs['labels'] = inputs['input_ids'].copy()
    return inputs

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/FutureIntern/results',
    evaluation_strategy='steps',  # Evaluate more frequently
    eval_steps=500,  # Evaluate every 500 steps
    num_train_epochs=3,  # Increase the number of epochs
    per_device_train_batch_size=8,  # Increase batch size if possible
    per_device_eval_batch_size=8,  # Increase evaluation batch size
    warmup_steps=1000,  # Increase warmup steps
    weight_decay=0.01,
    logging_dir='/content/drive/MyDrive/FutureIntern/logs',
    logging_steps=100,  # Log more frequently
    learning_rate=5e-5,  # Adjust learning rate
    gradient_accumulation_steps=2  # Accumulate gradients over 2 steps

)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]  # Add early stopping
)

# Train the model
trainer.train

# save the model and tokenizer explicitly
model_output_dir = '/content/drive/MyDrive/FutureIntern/results/model'

model.save_pretrained(model_output_dir)
tokenizer.save_pretrained(model_output_dir)

Using device: cpu


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


('/content/drive/MyDrive/FutureIntern/results/model/tokenizer_config.json',
 '/content/drive/MyDrive/FutureIntern/results/model/special_tokens_map.json',
 '/content/drive/MyDrive/FutureIntern/results/model/vocab.json',
 '/content/drive/MyDrive/FutureIntern/results/model/merges.txt',
 '/content/drive/MyDrive/FutureIntern/results/model/added_tokens.json',
 '/content/drive/MyDrive/FutureIntern/results/model/tokenizer.json')

In [36]:
from transformers import AutoTokenizer, AutoModelForCausalLM

def get_model_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    return total_params

def generate_text(prompt, tokenizer, model, max_length=50):

    # Define the parameters
    max_length=100  # Maximum length of the generated text
    temperature = 0.7  # Controls the randomness of predictions
    top_k = 50  # Limits the sampling pool to the top k most probable tokens
    top_p = 0.9  # Limits the sampling pool to the smallest set of tokens with a cumulative probability above p
    repetition_penalty = 1.2  # Penalizes repeated tokens
    num_beams = 5 #Beam Search
    no_repeat_ngram_size= 2

    # Tokenize the input text and set pad_token_id
    inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True)
    inputs['attention_mask'] = inputs['attention_mask']

    # Generate text
    outputs = model.generate(
        inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=max_length,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
        num_beams=num_beams,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        early_stopping=True,
        no_repeat_ngram_size=no_repeat_ngram_size,  # Prevents repeating n-grams
        repetition_penalty=repetition_penalty
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def main():
    # Load the tokenizer and model from the saved directory
    model_path = '/content/drive/MyDrive/FutureIntern/results/model'
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(model_path)

    # Calculate the Number of Parameters in the model being used for inference
    total_params = get_model_parameters(model)
    print(f"Total number of parameters: {total_params}")

    # Get input text from the user
    input_text = input("Enter the input text: ")

    # Generate Text
    generated_text = generate_text(input_text, tokenizer, model)
    # Display the output in a structured format

    print("\n--- Generated Text ---")
    print(f"Input: {input_text}")
    print(f"Output:\n{generated_text}")


    print("----------------------")

if __name__ == "__main__":
    main()




Total number of parameters: 124439808
Enter the input text: The quick brown fox jumps over the lazy dog and then…

--- Generated Text ---
Input: The quick brown fox jumps over the lazy dog and then…
Output:
The quick brown fox jumps over the lazy dog and then…

…runs off into the distance. The two of them are in the middle of the road, and they both look at each other like they're trying to figure out what's going on. They're both looking at the same thing, but they don't seem to know what it is. It's not like there's anything wrong with them, it's just that they can't figure it out. I'm not sure if they
----------------------
